# ETL Github exercise-solution

This is my solution for [Week_04's ETL-github exercise](Week_04/exercise_ETL-github.ipynb). 
The idea of the exercise was to workout a typical transformation from un-/semi-structured to structured data.
A typical ETL pipeline; We are not handling large and heterogeneous volumes of data, nor are we working with _data-lakes/warehouses_, but the whole idea and exercising the fundamental steps are there.

As a source of semi-/un-structured data we have GitHub: we are getting the metadata associated to some repositories -- and their "readme" files. GitHub repositories' metadata is retrieved through [GitHub's REST API](https://docs.github.com/en/rest/search#search-repositories). The API's response is in JSON format -- a _semi-structured_ data format; The _readme_ files -- when available -- are in _some_ textual data format (`.txt`, `.md`, `.rst`).

Eventually, we want tables representing such information (_ie_, the information got from GitHub).

To move such data from semi and unstructured data up to structured data demands (1) inspecting the source data in order to understand their basic structure, (2) designing the necessary transformations, (3) Python programming.

The exercise asks (explicitly) the following:
- [ ] Save each _items_ (_ie_, repository) metadata in a `.json` file in a directory named after _owner_ and/or _repository name_.
- [ ] Organize the (items) information among a "main", and "owners" and "license" and "topics" tables.
- [ ] Download the (repositories) corresponding _readme_ file (whenever there is one), and create an "index" table linking the most frequent words to the repositories we will find them.

The data request to GitHub is the following:
```
https://api.github.com/search/repositories?q=created:>2023-01-01+language:python&sort=stars&order=desc&per_page=100
```

Here is a reduced sample of the GitHub's response:
```json
{
  "total_count": 622770,
  "incomplete_results": false,
  "items": [
    {
      "id": 595893961,
      "owner": {
        "id": 19834515,
        "html_url": "https://github.com/lllyasviel",
        "site_admin": false
      },
      "html_url": "https://github.com/lllyasviel/ControlNet",
      "description": "Let us control diffusion models!",
      "created_at": "2023-02-01T02:43:30Z",
      "stargazers_count": 12522,
      "license": {
        "key": "apache-2.0",
        "node_id": "MDc6TGljZW5zZTI="
      },
      "topics": [
        
      ],
      "score": 1.0
    },
    {
      "id": 602270517,
      "owner": {
        "id": 125944572,
        "html_url": "https://github.com/FMInference",
        "site_admin": false
      },
      "html_url": "https://github.com/FMInference/FlexGen",
      "description": "Running large language models on a single GPU for throughput-oriented scenarios.",
      "created_at": "2023-02-15T21:18:53Z",
      "stargazers_count": 6490,
      "license": {
        "key": "apache-2.0",
        "node_id": "MDc6TGljZW5zZTI="
      },
      "topics": [
        "deep-learning",
        "gpt-3",
        "opt"
      ],
      "score": 1.0
    }
  ]
}
```

From which we can derive some conclusions about the structure:
- repositories' meta/data are inside the `items` list, and element of API response's top-level;
- each _item_ is a dictionary providing the repositories' primary data at its top-level;
- each _item_ has an `owner`, represented by a (flat) dictionary;
- each _item_ has a `license` dictionary (not entirely true, `license` may NOT exist, but we will ignore that now);
- each _item_ has a `topics` list, that may or may not be empty.

Regarding file and directory names, I'll use the same structural logics as GitHub and name directories after the _owner_ and files after the _repository_ names.

The process we'll go through to organize the data is basically the following:
- create "main" list, as well as the ancillary lists for "owners", "licenses", "topics"
- Loop over `items`:
  - copy the top-level key-values into a "main" object, push "main" object to a _main_ list of objects
  - save the repository `id` to be used by the nested data structures and relate entries in ancillary table
  - save the repository `name` for use in the (JSON and _readme_) file naming
  - copy `owner` key-values into a "owner" object, push it to _owners_ list together with the repository `id`
  - save the owner's `name` for use in the directory naming
  - copy `license` key-values into a "license" object, push it to _licenses_ list together with the repository `id`
  - copy `topics` values into a "topics" object, push it to _topics_ list together with the repository `id`

In [24]:
with open('token.txt','r') as fp:
    token = fp.read().strip()

## Workflow with Requests and built-ins

> The Python Standard Library provides the [`urllib`](https://docs.python.org/3/library/urllib.html) package for web requests; It is, though, a rather low-level interface, and it is broadly recommended to use the (external) [Requests](https://requests.readthedocs.io/) library instead.

Now that we have built the URL to request (in theory) exactly what we want from Github, all we have to do is to effectively get the (10) repositories data and then write it into JSON files as requested.

In [1]:
# Get data
import requests

response = requests.get(
    'https://api.github.com/search/repositories?q=created:>2023-01-01+language:python&sort=stars&order=desc&per_page=100', 
     headers={"Authorization": f"Bearer {token}"}
)

if response.status_code == 200:
    data_js = response.json()
    print(f"Request successful. Response items size: {len(data_js['items'])}")
else:
    print("Something went wrong with our request:", response.text)

Request successful. Response items size: 100


In [2]:
# Read 'items' out of response' json

items = tuple(
    data_js['items']
)

In [59]:
# Allocate data destinies
#
import json
from pathlib import Path

TEST = False

repos = []
owners = []
topics = []
licenses = []

word_freqs = []

base_dir = 'repos_data'
dir_name = '{owner_name}'
json_name = '{repo_name}.json'
readme_name = '{repo_name}.txt'


# Create the base directory, where everything will go
#
base_dir = Path(base_dir)
base_dir.mkdir(exist_ok=False)


def tokenize_readme(text):
    import string
    word_cnts = {}
    for line in text.split('\n'):
        line = line.strip()
        if not line:
            continue
        if line[0] in string.punctuation:
            continue
        for word in line.split():
            if len(word) < 3:
                continue
            if word in word_cnts:
                word_cnts[word] += 1
            else:
                word_cnts[word] = 1
                
    return sorted(word_cnts.items(), key=lambda pair: pair[1])[::-1]


# Go through data, select, and push them
#
for item in items:
    repo_id = item['id']
    repo_name = item['name']
    owner_name = item['owner']['login']

    # Copy data from the top-level keys
    #
    _repo = {}
    for k,v in item.items():
        if k in ('owner','license','topics'):
            continue
        if '_url' in k and k != 'html_url':
            continue
        _repo[k] = v
        
    assert 'id' in _repo, f"Oops: 'id' key not in repo {repo_name} record data"

    # Get data from "owner" dictionary
    _owner = {}
    for k,v in item['owner'].items():
        if '_url' in k and k != 'html_url':
            continue
        _owner[k] = v
        
    assert 'repo_id' not in _owner, "Error: there is alread a 'repo_id' key in owner record"
    
    # Make the link/relation between owner and repository
    _owner['repo_id'] = repo_id

    # License may not be defined, but if it is, we get everything from it
    #
    _license = None
    if 'license' in item and item['license']:
        _license = {k:v for k,v in item['license'].items()}
        _license['repo_id'] = repo_id
        
    # Topics may be empty
    #
    _topics = None
    if item['topics']:
        _topics = [{'repo_id': repo_id, 'topic': _topic} for _topic in item['topics']]

    repos.append(_repo)
    owners.append(_owner)
    if _license:
        licenses.append(_license)
    if _topics:
        topics.extend(_topics)

    # Let's create the output files for this repository.
    # First, we create the repo/owner directory
    #
    _owner_dir = base_dir / Path(dir_name.format(owner_name=owner_name))
    _owner_dir.mkdir(exist_ok=True)
    
    # Define the output JSON file
    #
    _json_filename = _owner_dir / json_name.format(repo_name=repo_name)

    if TEST:
        print(f"Content of item '{owner_name}/{repo_name}' would be written to '{_json_filename}'")
    else:
        with open(_json_filename, 'w') as fp:
            json.dump(item, fp)


    # Before we go, let's download the readme files. No processing will be done now, will just save in dir.
    # - according to GitHub docs, we can get the appropriate "readme" content directly from the API
    #   https://docs.github.com/en/rest/repos/contents#get-a-repository-readme
    #
    _readme_filename = _owner_dir / readme_name.format(repo_name=repo_name)
    try:
        r = requests.get(f"{_repo['url']}/readme", 
                 headers={"Accept": "application/vnd.github.raw",
                          "Authorization": f"Bearer {token}"})
        _readme_content = r.text
        
        _freqs = tokenize_readme(_readme_content)[:100]
        _findx = [{'repo_id': repo_id, 'word': _word, 'freq': _freq} for _word,_freq in _freqs]
        word_freqs.extend(_findx)
        
        with open(_readme_filename, 'w') as fp:
            fp.write(_readme_content)
            
    except:
        print("Readme content not found")
        _index = None
        
    if TEST:
        break

In [60]:
# Check if our final data is minimally consistent

assert len(repos) == len(owners)
assert len(repos) == len(items)

In [61]:
# How does 'repos', 'owners', 'licenses', 'topics' look like?

from pprint import pprint

pprint(repos[0])

{'allow_forking': True,
 'archived': False,
 'created_at': '2023-02-01T02:43:30Z',
 'default_branch': 'main',
 'description': 'Let us control diffusion models!',
 'disabled': False,
 'fork': False,
 'forks': 1007,
 'forks_count': 1007,
 'full_name': 'lllyasviel/ControlNet',
 'has_discussions': True,
 'has_downloads': True,
 'has_issues': True,
 'has_pages': False,
 'has_projects': True,
 'has_wiki': True,
 'homepage': '',
 'html_url': 'https://github.com/lllyasviel/ControlNet',
 'id': 595893961,
 'is_template': False,
 'language': 'Python',
 'name': 'ControlNet',
 'node_id': 'R_kgDOI4SeyQ',
 'open_issues': 145,
 'open_issues_count': 145,
 'private': False,
 'pushed_at': '2023-03-08T08:41:14Z',
 'score': 1.0,
 'size': 125302,
 'stargazers_count': 12548,
 'updated_at': '2023-03-09T17:02:17Z',
 'url': 'https://api.github.com/repos/lllyasviel/ControlNet',
 'visibility': 'public',
 'watchers': 12548,
 'watchers_count': 12548,
 'web_commit_signoff_required': False}


In [62]:
pprint(owners[0])

{'gravatar_id': '',
 'html_url': 'https://github.com/lllyasviel',
 'id': 19834515,
 'login': 'lllyasviel',
 'node_id': 'MDQ6VXNlcjE5ODM0NTE1',
 'repo_id': 595893961,
 'site_admin': False,
 'type': 'User',
 'url': 'https://api.github.com/users/lllyasviel'}


In [63]:
pprint(licenses[0])

{'key': 'apache-2.0',
 'name': 'Apache License 2.0',
 'node_id': 'MDc6TGljZW5zZTI=',
 'repo_id': 595893961,
 'spdx_id': 'Apache-2.0',
 'url': 'https://api.github.com/licenses/apache-2.0'}


In [64]:
pprint(topics[0])

{'repo_id': 602270517, 'topic': 'deep-learning'}


In [65]:
pprint(word_freqs[0])

{'freq': 66, 'repo_id': 595893961, 'word': 'the'}


# Create tables

In [66]:
import pandas

In [67]:
df_repos = pandas.DataFrame(repos)
df_owners = pandas.DataFrame(owners)
df_topics = pandas.DataFrame(topics)
df_licenses = pandas.DataFrame(licenses)

In [68]:
df_repos.head()

id       node_id                 name  \
0  595893961  R_kgDOI4SeyQ           ControlNet   
1  601538369  R_kgDOI9q_QQ                llama   
2  602270517  R_kgDOI-XrNQ              FlexGen   
3  600798098  R_kgDOI89zkg  sd-webui-controlnet   
4  586711486  R_kgDOIviBvg     instruct-pix2pix   

                        full_name  private  \
0           lllyasviel/ControlNet    False   
1          facebookresearch/llama    False   
2             FMInference/FlexGen    False   
3    Mikubill/sd-webui-controlnet    False   
4  timothybrooks/instruct-pix2pix    False   

                                            html_url  \
0           https://github.com/lllyasviel/ControlNet   
1          https://github.com/facebookresearch/llama   
2             https://github.com/FMInference/FlexGen   
3    https://github.com/Mikubill/sd-webui-controlnet   
4  https://github.com/timothybrooks/instruct-pix2pix   

                                         description   fork  \
0                   Let us control diffusion models!  False   
1                    Inference code for LLaMA models  False   
2  Running large language models on a single GPU ...  False   
3                     WebUI extension for ControlNet  False   
4                                               None  False   

                                                 url            created_at  \
0  https://api.github.com/repos/lllyasviel/Contro...  2023-02-01T02:43:30Z   
1  https://api.github.com/repos/facebookresearch/...  2023-02-14T09:29:12Z   
2   https://api.github.com/repos/FMInference/FlexGen  2023-02-15T21:18:53Z   
3  https://api.github.com/repos/Mikubill/sd-webui...  2023-02-12T16:26:27Z   
4  https://api.github.com/repos/timothybrooks/ins...  2023-01-09T03:14:31Z   

   ... open_issues_count allow_forking is_template  \
0  ...               145          True       False   
1  ...               118          True       False   
2  ...                24          True       False   
3  ...               151          True       False   
4  ...                29          True       False   

   web_commit_signoff_required  visibility  forks open_issues  watchers  \
0                        False      public   1007         145     12548   
1                        False      public   1388         118      9356   
2                        False      public    332          24      6496   
3                        False      public    434         151      4363   
4                        False      public    332          29      3682   

   default_branch  score  
0            main    1.0  
1            main    1.0  
2            main    1.0  
3            main    1.0  
4            main    1.0  

[5 rows x 36 columns]

In [69]:
df_owners.head()

login         id                           node_id gravatar_id  \
0        lllyasviel   19834515              MDQ6VXNlcjE5ODM0NTE1               
1  facebookresearch   16943930  MDEyOk9yZ2FuaXphdGlvbjE2OTQzOTMw               
2       FMInference  125944572                      O_kgDOB4HC_A               
3          Mikubill   31246794              MDQ6VXNlcjMxMjQ2Nzk0               
4     timothybrooks   10535711              MDQ6VXNlcjEwNTM1NzEx               

                                             url  \
0        https://api.github.com/users/lllyasviel   
1  https://api.github.com/users/facebookresearch   
2       https://api.github.com/users/FMInference   
3          https://api.github.com/users/Mikubill   
4     https://api.github.com/users/timothybrooks   

                              html_url          type  site_admin    repo_id  
0        https://github.com/lllyasviel          User       False  595893961  
1  https://github.com/facebookresearch  Organization       False  601538369  
2       https://github.com/FMInference  Organization       False  602270517  
3          https://github.com/Mikubill          User       False  600798098  
4     https://github.com/timothybrooks          User       False  586711486

In [70]:
df_topics.head()

repo_id                  topic
0  602270517          deep-learning
1  602270517                  gpt-3
2  602270517        high-throughput
3  602270517  large-language-models
4  602270517       machine-learning

In [71]:
df_licenses.head()

key                             name      spdx_id  \
0  apache-2.0               Apache License 2.0   Apache-2.0   
1     gpl-3.0  GNU General Public License v3.0      GPL-3.0   
2  apache-2.0               Apache License 2.0   Apache-2.0   
3         mit                      MIT License          MIT   
4       other                            Other  NOASSERTION   

                                          url           node_id    repo_id  
0  https://api.github.com/licenses/apache-2.0  MDc6TGljZW5zZTI=  595893961  
1     https://api.github.com/licenses/gpl-3.0  MDc6TGljZW5zZTk=  601538369  
2  https://api.github.com/licenses/apache-2.0  MDc6TGljZW5zZTI=  602270517  
3         https://api.github.com/licenses/mit  MDc6TGljZW5zZTEz  600798098  
4                                        None  MDc6TGljZW5zZTA=  586711486

In [74]:
df_index = pandas.DataFrame(word_freqs)
df_index

repo_id         word  freq
0     595893961          the    66
1     595893961   ControlNet    23
2     595893961          for    20
3     595893961          and    20
4     595893961      Prompt:    19
...         ...          ...   ...
8967  584547922      denoise     1
8968  584547922        large     1
8969  584547922  opportunity     1
8970  584547922         have     1
8971  584547922          Now     1

[8972 rows x 3 columns]

### Finally, write those tables down

In [75]:
df_repos.to_csv('repos.csv', index=False)
df_owners.to_csv('owners.csv', index=False)
df_topics.to_csv('topics.csv', index=False)
df_licenses.to_csv('licenses.csv', index=False)

df_index.to_csv('index.csv', index=False)